# RUN one sample in a run

In [1]:
from prompt_manager import get_k_shot_with_answer, view_instruction, row_instruction
import pandas as pd
from utils import parse_specific_composition
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from sqlalchemy import create_engine
from executor import SQLManager
import sqlparse

### 设置全局变量
这里会设置任务名和split
tabfact：事实判断数据集，这里使用small_test=True，选取测试集合中的2000+条数据

In [2]:
task_name = 'tabfact'
split = 'test'
model_name = 'gpt-3.5-turbo-0125'
model = ChatOpenAI(model_name=model_name, openai_api_base="https://api.chatanywhere.com.cn/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.01)
schema_information = pd.read_csv(f"result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine=engine)

In [3]:
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)


### 数据可视化

In [14]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
#选取数据集对应的index
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
i = 66

def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['table']['id'])
    print(data['statement'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])
print(table_loader.dataset[i]['table']['caption'])
show_table(table_loader.dataset[i]) 

1979 world figure skating championships
2-11312764-4.html.csv
linda fratianne finish in first place , but anett pötzsch do not manage to get in the top 3
+--------+-----------------------------+----------------+-----------+----------+----------+
|   rank | name                        | nation         |   sp_+_fs |   points |   places |
|--------+-----------------------------+----------------+-----------+----------+----------|
|      1 | linda fratianne             | united states  |         1 |   186.92 |       11 |
|      2 | anett pötzsch               | east germany   |         3 |   184.36 |       18 |
|      3 | emi watanabe                | japan          |         4 |   180.52 |       31 |
|      4 | dagmar lurz                 | west germany   |         6 |   179.96 |       33 |
|      5 | denise biellmann            | switzerland    |         2 |   177.28 |       49 |
|      6 | lisa - marie allen          | united states  |         5 |   176.68 |       54 |
|      7 | claudia

### step-back & decompose prompt

In [5]:
# decompose prompt
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
    # "after 2005 , the winner of the lifetime achievement award be andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle",
                  "all 12 club play a total of 22 game for the wru division one east",
                #   "a gamecube game loss the award in each of the first 3 year",
                "from 1980 to 2011 , apoel bc lose more than 2 time as many game as it win",
                  "polona hercog 1890partner with alberta brianti after she have stephanie vogt as the partner",
                  ]
task_examples = ["query rewrite", "query decompose", "query ambiguity resolve"]
new_query_examples = [
    # "Who were the winners of the lifetime achievement award after 2005?;",
                      "How many clubs play for the wru division one east in total?; How many clubs play 22 game for the wru division one east?;",
                    #   "a gamecube game loss the award in each of the first 3 year",
                    "from 1980 to 2011 , how many games did apoel bc lose?; from 1980 to 2011 , how many games did apoel bc win?;",
                      "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      ]
num_k = 3
inds = [1, 124, 5]
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Query: {query}
Sub-Table: {table}
new_query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub-queries. Based on the table, provide at most 2 continuity sub-queries for knowledge that you need. 
Split the queries with ’;’.""",
    suffix=
    """Query: {query}
Sub-Table: {table}
new_query: 
    """,
    input_variables=["query", "table"],
)

# Sub-questions are separated by semicolons.
# answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
#                                     template="""
# Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Query: {claim}
# answer the question given in the query. Only return the string instead of other format information. Do not repeat the question.
# """ )


In [6]:
#step-back prompt
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
inds = [8, 15]
num_k = 2
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]
new_query_examples = [
    # "Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
    "which college list be public?",
    "which game be all score over time?"
    ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Query: {query}
Sub-Table: {table}
new_query: {new_query}""")

examples_dict = [{"query": table_loader.dataset[inds[i]]['statement'],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Based on the table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """
Query: {query}
Sub-Table: {table}
    """,
    input_variables=["query", "table"],
)


In [ ]:
#简单测试prompt的脚本，可以不运行这个block

# table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
# i = 90
# sample = table_loader.normalize_table(table_loader.dataset[i])
# all_queries = []
# formatter = TableFormat(format='none', data=sample, use_sampling=True)
# llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=True)
# batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)

In [6]:
# final inference prompt（deprecated）


# def get_k_shot_with_answer(k: int=1):
#     sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"]
#     thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. Therefore, the claim that 2 is the fewest points they received is false. The output should be 0."]
#     tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
#     claims = ["2 be the fewest point that roger dutton / tony wright receive"]
#     # inds from test split
#     examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
#     """
#     SQL Excuted: 
#     ```{SQL}```
#     Sub-table: {table}
#     ...

#     Query: {claim}
#     Thought: {thought}
#     Output: {output}
#     """)
#     examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '0']))
#     prompt_template = FewShotPromptTemplate(
#         examples=[examples_dict],
#         example_prompt=examples_prompt,
#         prefix="""Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. Please think step by step and return 0 or 1 without any other information at last.""",
#         suffix=
#         """
#     {information}
#     Query: {query}
#     Thought: """,
#         input_variables=["table", "query"],
# )
#     return prompt_template

In [7]:
# column sampling prompt

def get_k_shot_with_aug(k: int=2):
    table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)

    inds = [3, 6, 260, 33]
    Output_examples = [
                       'team, goals_for',
                       'year, game, platform_s',
                       'name, population_density_km_2_, population_2011_census_'
                       'leading_scorer, score, date']
    linking_examples = ['the team -> team; the most goal for -> goals_for',
                        'gamecube -> platform_s; gamecube game -> game; the first 3 year -> year;',
                        'alberta -> name; population density -> population_density_km_2_; 4257744 less people -> population_2011_census_; 2011 -> population_2011_census_'
                        'jason richardson -> leading_scorer; leading scorer -> score; month -> date; 23 point per game -> leading_scorer'
    ]
    examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
    """
    Table: {table}
    Query: {claim}
    Column linking: {linking}
    Columns: {output}""")
    num_k = 3
    examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_nl_sep(table_loader.dataset[inds[i]]['table']['caption']),
                                        "claim": table_loader.dataset[inds[i]]['statement'],
                                        "linking": linking_examples[i],
                                        # "summary": summary_examples[i],
                                        "output": Output_examples[i]} for i in range(num_k)]
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
    Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the table.
    Approach this task as follows，read the claim thoroughly and list every possible link from query term to column in Table. Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.""",
    # You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
    # Given the following table and query, you should output columns related to the query or contain useful information about the query. 
    # Here are some examples:""",
        suffix=
        """
    Table: {table}
    Query: {claim}
    """,
        input_variables=["table", "claim"],
)
    return prompt_template


定义两个函数：

**scene_A**: 执行column sampling, SQL生成得到子表，最终返回sql和子表，不回答问题


**scene_B**: 执行column sampling, SQL生成执行并直接返回问题回答的结果

In [8]:
from utils import parse_output

def scene_A(query, sample, verbose=True):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['id']]['summary'], aug_information.loc[sample['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]
        formatter.normalize_schema(schema_information.loc[sample['id']]['schema'])
        try: 
            formatter.data = formatter.data.loc[:, columns]
            formatter.all_data = formatter.all_data.loc[:, columns]
        except:
            pass
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + extra_information
                                            })], return_only_outputs=True)[0]['text']
    print("total_tokens:", cb.total_tokens)
    print(stage_2_batch_pred)
    # stage 3: SQL Excution
    try: 
        formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
    except:
        formatter.data = formatter.all_data
        stage_2_batch_pred = 'SELECT * from DF;'
    if len(formatter.data) == 0:
        return query, stage_2_batch_pred, 'No data from database', cb.total_tokens
    return query, stage_2_batch_pred, formatter.format_html(), cb.total_tokens

In [9]:
from utils import parse_output
answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
                                    template="""
Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
SQL Excuted: 
```{SQL}```
Sub-table: 
{table}
Query: {claim}
Please provide a clear, concise statement in response to the question. If you cannot answer the question based on the sub-table, just say 'Cannot get answer from sub-table'
""" )
def scene_B(query, sample, verbose=False):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['id']]['summary'], aug_information.loc[sample['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            # 'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]
        
        try: 
            formatter.data = formatter.data.loc[:, columns]
            formatter.all_data = formatter.all_data.loc[:, columns]
        except:
            pass
        formatter.normalize_schema(schema_information.loc[sample['id']]['schema'])
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + extra_information
                                            })], return_only_outputs=True)[0]['text']
    
        
        print(stage_2_batch_pred)
        # stage 3: SQL Excution
        try: 
            formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
        except:
            formatter.data = formatter.all_data
            stage_2_batch_pred = 'SELECT * from DF;'
        llm_chain = LLMChain(llm=model, prompt=answer_instruction, verbose=verbose)
        response = llm_chain.batch([dict({'table': formatter.format_html(),
                                                'claim': query,
                                                'SQL':  stage_2_batch_pred
                                                })], return_only_outputs=True)[0]['text']
    print("total_tokens:", cb.total_tokens)
    return response, cb.total_tokens

In [10]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

In [11]:
from typing import List
import os
import json
def save_csv(input_list: List[List], label_list: List, file_path):
    import pandas as pd
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    assert len(input_list) == len(label_list)
    df = pd.DataFrame()
    for i in range(len(label_list)):
        df[label_list[i]] = pd.Series(input_list[i])
    if os.path.exists(file_path) and file_path.endswith('.csv'):
        df_origin = pd.read_csv(file_path)
        df = pd.concat([df_origin, df], axis=0)
    df.to_csv(file_path, index=False, encoding='utf-8')

In [12]:
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

### RUN

In [13]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime

table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.com.cn/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.01)
save_path = f"result/answer/tabfact_zh_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
# template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
template = """
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.

SQL Excuted: 
```{SQL}```
Sub-table:
{table}
Extra information:
{information}

Query: {query}
Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
tokens = []
outputs = []
labels = []
# muilti_answer_instruction = get_k_shot_with_answer()

i = 66
sample = table_loader.normalize_table(
                    table_loader.dataset[i])
all_tokens = 0
labels.append(sample['label'])
all_queries = []
formatter = TableFormat(format='none', data=sample, use_sampling=True)
with get_openai_callback() as cb:
    llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
    batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)
    all_queries.append(batch_pred[0]['text'].split(':')[-1])
    llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
    batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)
    all_queries.extend(batch_pred[0]['text'].split(';'))
    print(all_queries)
all_tokens += cb.total_tokens
args_list = [{"query": q, "sample": sample} for q in all_queries]
ans_from_B = parallel_run_kwargs(scene_B, args_list)
results = [res[0] for res in ans_from_B if res[0] != 'Cannot get answer from sub-table']
all_tokens += sum([res[1] for res in ans_from_B])
#With answer
with get_openai_callback() as cb:
    imp_input = scene_A(sample['query'], sample, False)
    llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
    batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
print(batch_pred[0])
all_tokens += cb.total_tokens
print('ALL TOKENS', all_tokens)
outputs.append(batch_pred[0]['text'])
# if (i + 1) % 10 == 0:
#     save_csv([outputs, labels], ['preds', 'labels'], file_path=save_path)
#     outputs = []
#     labels = []
#With no answer
# temp = [f"""
# SQL Excuted for extra information: 
# ```{res[1]}```
# Sub-table for extra information: {res[2]}""" for res in results if res[2] != 'No data from database']
# imp_input = scene_A(sample['query'], sample, False)
# llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
# batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(temp)}], return_only_outputs=True)
# print(batch_pred[0])
# outputs.append(batch_pred[0]['text'])


[' who finish in first place and who do not manage to get in the top 3?', 'How did Linda Fratianne finish in the competition?', ' Did Anett Pötzsch manage to get in the top 3 in the competition?']
Column linking: Linda Fratianne -> name; finish in the competition -> rank
Columns: name, rank
Column linking: first place -> rank 1; top 3 -> rank 1, rank 2, rank 3
Columns: rank, name
SELECT rank
FROM DF
WHERE name = 'Linda Fratianne';
SELECT name
FROM DF
WHERE rank = 1 OR rank > 3;
total_tokens: 1528
Column linking: Anett Pötzsch -> name; top 3 -> rank
Columns: name, rank
total_tokens: 2102
SELECT name, rank
FROM DF
WHERE name = 'Anett Pötzsch' AND rank <= 3;
total_tokens: 1567
Column linking: linda fratianne -> name; first place -> rank; anett pötzsch -> name; top 3 -> rank
Columns: name, rank
total_tokens: 1407
SELECT name, rank
FROM DF
WHERE name = 'linda fratianne' OR name = 'anett pötzsch';


> Entering new LLMChain chain...
Prompt after formatting:

Below is a sub-table generated by 

In [15]:
#前100条我测试的badcase， ind列表
[1, 11, 14, 22, 26, 37, 38, 40, 47, 49, 50,  59, 63, 65, 66, 68, 82, 88, 99]

20

### some sample filtereed

In [ ]:
46: west berkshire brewery 's maggs magnificent mild be its most decorate beer between 1995 and 2009

In [29]:
20：Based on the information provided, the SQL query filtered for records where the place starts with 't9' and the country is 'united states'. The result of the query shows that there are 3 people who meet these criteria. However, the extra information states that there were actually 4 people who tied for ninth place, and all of them were from the United States.\n\nTherefore, the provided claim/query is false. The correct number of people from the United States who tied for ninth place is 4, not 3. \n\nFinal answer: 0
88: To verify whether the term start for Bashkim Fino is after the term start for Vilson Ahmeti, we need to compare the dates mentioned in the sub-table. From the sub-table:\n- Vilson Ahmeti's term started on December 10, 1991.\n- Bashkim Fino's term started on March 11, 1997.\n\nSince December comes before March in the calendar year, it is evident that Vilson Ahmeti's term started before Bashkim Fino's term. Therefore, the claim that the term start for Bashkim Fino is after the term start for Vilson Ahmeti is FALSE.\n\nHence, the answer is 0
72: To verify the claim that the gap between the first and last player being a total of 58.04 is true, we need to calculate the difference between the points of the first and last player based on the given information.\n\nGiven:\n- Total gap between the first and last rank is 18.\n- Rank of the first player is 1.\n- Rank of the last player is 19.\n\nLet's calculate the points difference between the first and last player:\npoints difference = (points of last player) - (points of first player)\n\nSince the total gap between the first and last rank is 18, we can calculate the points difference as follows:\npoints difference = 18 * (MAX(points) - MIN(points))\n\nGiven that the calculated point gap from the sub-table is 58.04, we can substitute this value into the formula:\n18 * 58.04 = 1044.72\n\nTherefore, the claim that the gap between the first and last player being a total of 58.04 is false.\n\nFinal answer: 0
76: The SQL query filters for competitors from France with a rank less than 5. Since the sub-table generated from the query shows no data, it means that there were no competitors from France who finished better than 5th place. \n\nTherefore, the provided claim that France's competitors all finished better than 5th place is TRUE.\n\nFinal answer: 1

SyntaxError: invalid character '：' (U+FF1A) (3998830562.py, line 1)

In [ ]:
def str_normalize(user_input, recognition_types=None):
    """A string normalizer which recognize and normalize value based on recognizers_suite"""
    user_input = str(user_input)
    user_input = user_input.replace("\\n", "; ")

    def replace_by_idx_pairs(orig_str, strs_to_replace, idx_pairs):
        assert len(strs_to_replace) == len(idx_pairs)
        last_end = 0
        to_concat = []
        for idx_pair, str_to_replace in zip(idx_pairs, strs_to_replace):
            to_concat.append(orig_str[last_end : idx_pair[0]])
            to_concat.append(str_to_replace)
            last_end = idx_pair[1]
        to_concat.append(orig_str[last_end:])
        return ''.join(to_concat)

    if recognition_types is None:
        recognition_types = [
            "datetime",
            "number",
            "ordinal",
            "percentage",
            "age",
            "currency",
            "dimension",
            "temperature",
        ]
    culture = Culture.English
    for recognition_type in recognition_types:
        if re.match("\d+/\d+", user_input):
            # avoid calculating str as 1991/92
            continue
        recognized_list = getattr(
            recognizers_suite, "recognize_{}".format(recognition_type)
        )(
            user_input, culture
        )  # may match multiple parts
        strs_to_replace = []
        idx_pairs = []
        for recognized in recognized_list:
            if not recognition_type == 'datetime':
                recognized_value = recognized.resolution['value']
                if str(recognized_value).startswith("P"):
                    # if the datetime is a period:
                    continue
                else:
                    strs_to_replace.append(recognized_value)
                    idx_pairs.append((recognized.start, recognized.end + 1))
            else:
                if recognized.resolution:  # in some cases, this variable could be none.
                    if len(recognized.resolution['values']) == 1:
                        strs_to_replace.append(
                            recognized.resolution['values'][0]['timex']
                        )  # We use timex as normalization
                        idx_pairs.append((recognized.start, recognized.end + 1))

        if len(strs_to_replace) > 0:
            user_input = replace_by_idx_pairs(user_input, strs_to_replace, idx_pairs)

    if re.match("(.*)-(.*)-(.*) 00:00:00", user_input):
        user_input = user_input[: -len("00:00:00") - 1]
        # '2008-04-13 00:00:00' -> '2008-04-13'
    return user_input